In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [3]:
# Load the dataset
df = pd.read_csv('../data/sensor_validation.csv', index_col='timestamp', parse_dates=True)

In [4]:
# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'sensor_15'])

In [5]:
# Interpolate missing values
df.interpolate(method='time', inplace=True)

C:\Users\ksfach\AppData\Local\Temp\ipykernel_13248\734182364.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='time', inplace=True)


In [6]:
# Normalize the sensor readings
sensor_columns = df.columns[:-1]  # Exclude the 'machine_status' column
scaler = StandardScaler()
df[sensor_columns] = scaler.fit_transform(df[sensor_columns])

C:\Users\ksfach\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\ksfach\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\ksfach\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [7]:
# Function to create sequences
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
    return np.array(sequences)

In [8]:
# Parameters
sequence_length = 50  # Length of the sequence for LSTM

# Prepare the data
sensor_data = df[sensor_columns].values
sequences = create_sequences(sensor_data, sequence_length)

In [9]:
# Split the data into training and test sets
X_train, X_test = train_test_split(sequences, test_size=0.2, random_state=42)

In [1]:
import keras as ks
print(ks.__version__)

2.15.0


In [10]:
from keras.models import load_model

# Path to your model in Google Drive
model_path = '../models/lstm_autoencoder.keras'

# Load the model
model = load_model(model_path)

ValueError: Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['lstm/lstm_cell/kernel:0', 'lstm/lstm_cell/recurrent_kernel:0', 'lstm/lstm_cell/bias:0']

In [ ]:
# Detect anomalies
X_test_pred = model.predict(X_test)
test_mse_loss = np.mean(np.power(X_test - X_test_pred, 2), axis=(1, 2))

1377/1377 [==============================] - 125s 88ms/step


In [ ]:
# Determine threshold for anomalies
threshold = np.percentile(test_mse_loss, 99)  # 95th percentile

NameError: name 'np' is not defined

In [ ]:
# Predict anomalies
test_score_df = pd.DataFrame(index=df.index[sequence_length + len(X_train):])
test_score_df['loss'] = test_mse_loss
test_score_df['threshold'] = threshold
test_score_df['anomaly'] = test_score_df['loss'] > test_score_df['threshold']